In [1]:
import torch
import transformers
import pickle as pkl
import numpy as np
import sys
path = "../"
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    path = "./drive/MyDrive/566 Project"
    from google.colab import drive
    drive.mount('/content/drive')

In [6]:
clean_str = "ILSVRC2012_val_00000001_n01751748.JPEG"
c_id = "_".join(clean_str.split("_")[2:4])
c_id = c_id.split(".")[0]

# noise_str = "ILSVRC2012_val_00000262.JPEG"
# n_id = noise_str.split("_")[2].split(".")[0]
# c_id, n_id


In [13]:
data = open(path+"data/data1_cropped_name", 'rb')
data1 = pkl.load(data)
data = open(path+"data/data2_cropped_name", 'rb')
data2 = pkl.load(data)
data = data1+data2

In [9]:
# load noisey and gaussian data
import torchvision.transforms as trn
import os
import numpy as np
from PIL import Image

# directory = "./noise/gaussian_noise"

# total_files = 0
# stop = 30000

def pic_id(fn, clean=True):
    if clean:
        c_id = "_".join(fn.split("_")[2:4])
        return c_id.split(".")[0]
    else:
        return fn.split("_")[2].split(".")[0]

# use_img = False

# noise = {}
# for dirname in os.listdir(directory):
#     if dirname == ".DS_Store": continue
#     dir_ = os.path.join(directory, dirname)
#     if dirname == "1":
#         for subdir in os.listdir(dir_):
#             if subdir == ".DS_Store": continue
#             subsubdir = os.path.join(dir_, subdir)
#             for subfile in os.listdir(subsubdir):
#                 total_files += 1
#                 # if total_files < stop/2: continue
#                 if subfile == ".DS_Store": continue
#                 if total_files == stop: break
#                 path_ = os.path.join(subsubdir, subfile)
#                 img = np.asarray(Image.open(path_))
#                 for d in data:
#                     if np.array_equal(d[0], img):
#                         use_img = True
#                         break
#                 if use_img:
#                     name = pic_id(subfile, False) + "_" + subdir
#                     noise[name] = img
#                     print(name)
#                 use_img = False


# use_img = False

directory = path+"data/clean"

total_files = 0

clean = {}
data_transforms = trn.Compose([trn.Resize(256), trn.CenterCrop(224)])

for fn in os.listdir(directory):
    # if total_files == stop: break
    if fn == ".DS_Store": continue
    total_files += 1
    # if total_files < stop/2: continue
    subdir = os.path.join(directory, fn)
    try:
        img = Image.open(subdir)
        img =  np.asarray(data_transforms(img))
#         for d in data:
#             if np.array_equal(d[1], img):

#                 use_img = True
#                 break
        name = pic_id(fn)
        clean[name] = img
        print(name)
    except:
        print(subdir)
        continue

00029141_n03887697
00044583_n03481172
00003083_n03724870
00015305_n04040759
00018341_n04550184
00029021_n04285008
00028840_n02883205
00007560_n06359193
00008545_n02280649
00012537_n07693725
00021683_n01675722
00048315_n02105855
00041671_n04090263
00046157_n02963159
00011451_n04487081
00045833_n11939491
00021645_n02497673
00005071_n03457902
00002686_n07871810
00030929_n02123597
00025772_n02787622
00004580_n01768244
00047972_n01872401
00005932_n01776313
00033959_n03042490
00007001_n02108089
00024082_n02233338
00049629_n02117135
00044259_n02992211
00027077_n02091032
00032585_n02497673
00028517_n04228054
00047354_n01687978
00041228_n04127249
00012429_n01770393
00024531_n02206856
00026799_n03787032
00013268_n03661043
00043368_n03710193
00006195_n01514859
00009268_n03785016
00030866_n02790996
00015650_n01930112
00005376_n07579787
00003590_n03216828
00041029_n02066245
00037034_n04592741
00030457_n07718472
00011559_n04579432
00039027_n02093991
00002909_n02951585
00032730_n02328150
00037882_n01

In [23]:
total_files

50988

In [10]:

# pair up clean data

data = [] # [[filename, adv_0, clean_0], ..., [filename, adv_k, clean_k]]

for k,v in clean.items():
    data.append([k, v])

In [11]:
len(data)

50988

In [13]:
len(clean.items()) # , len(noise.items())

50988

In [15]:
# data[0]

In [16]:
import pickle as pkl
fileObject = open("test_clean_cropped_name", 'wb')
pkl.dump(data, fileObject)
fileObject.close()

# fileObject = open("test2_cropped_name", 'wb')
# pkl.dump(data[4000:], fileObject)
# fileObject.close()

In [17]:
ls

'MAE Finetune.ipynb'         Old                          finetune.ipynb
'MAE Hugging Face'           ViT_Image_Classifier.ipynb
 MakeReconstructions.ipynb   clean1_cropped_name


In [9]:
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms

class DS(Dataset):
     def __init__(self, data):
         super(Dataset, self).__init__()
         self.data = data
        
     def __len__(self):
         return len(data)*2
        
     def __getitem__(self, index):
        assert self.__len__ > index
        return self.data[index][0], self.data[index][1]

ds = DS(data)
train_size = int(0.9 * len(ds))
test_size = len(ds) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(ds, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

<module 'transformers.models' from '/Users/alexkorman/miniconda3/envs/ml/lib/python3.9/site-packages/transformers/models/__init__.py'>

In [4]:

import transformers
from transformers import ViTMAEForPreTraining, AutoImageProcessor

# image_processor = AutoImageProcessor.from_pretrained('facebook/vit-mae-large')
# model = ViTMAEForPreTraining.from_pretrained('facebook/vit-mae-large')


ImportError: cannot import name 'AutoImageProcessor' from 'transformers' (/Users/alexkorman/miniconda3/envs/ml/lib/python3.9/site-packages/transformers/__init__.py)

In [ ]:
# chkpt_dir = 'mae_visualize_vit_large_ganloss.pth'
from util import prepare_model
model = prepare_model(model, 'mae_visualize_vit_large_ganloss.pth')
# the forward pass runs through the encoder, decoder, and forward loss, which compares the output to the actual image.
# we're going to ignore the OG preturbed and compare it to the purified one


In [ ]:
# same as og model https://github.com/facebookresearch/mae/blob/main/models_mae.py
def forward_loss(self, imgs, pred, mask):
        """
        imgs: [N, 3, H, W]
        pred: [N, L, p*p*3]
        mask: [N, L], 0 is keep, 1 is remove, 
        """
        target = self.patchify(imgs)
        if self.norm_pix_loss:
            mean = target.mean(dim=-1, keepdim=True)
            var = target.var(dim=-1, keepdim=True)
            target = (target - mean) / (var + 1.e-6)**.5

        loss = (pred - target) ** 2
        loss = loss.mean(dim=-1)  # [N, L], mean loss per patch

        loss = (loss * mask).sum() / mask.sum()  # mean loss on removed patches
        return loss


In [ ]:
from torch import optim, nn


device = torch.device("mps")
# Prob1-2
epochs = 8
learning_rate = 1e-3
# device = "cpu"
model = model.train()   # set model in train mode (eg batchnorm params get updated)
# build optimizer and loss function
criterion = nn.BCELoss().to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

#################################### END TODO #######################################

train_it = 0
rec_loss = 0
batch_size = 8
in_size = out_size = 224
for ep in range(epochs):
  print("Run Epoch {}".format(ep))

  for img, _ in train_set: # loads a batch of data
    inputs = img.reshape([batch_size, in_size]).to(device)
    
    optimizer.zero_grad()

    outputs = model(**inputs)

    loss = criterion(outputs, inputs.reshape(batch_size, 1, in_size))

    rec_loss = loss
    
    loss.backward()

    optimizer.step()

    if train_it % 100 == 0:
      print("It {}: Reconstruction Loss: {}".format(train_it, rec_loss))
    train_it += 1
  # #################################### END TODO #####################################

print("Done!")
